In [1]:
import sys
import os
import keras_tuner
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Layer, Dropout
from tensorflow.keras import layers
from tensorflow.keras.layers import PReLU


# Get the absolute path of the current script's directory
current_dir = os.path.dirname(os.path.abspath("transformer0.ipynb"))

# Get the absolute path of the parent directory (project_folder)
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to the Python path
sys.path.append(parent_dir)

from FNN1_1 import baseline_deviation, baeline_out_deviation, baseline_long_deviation, baseline_relError, absSum
baseline_out_deviation = baeline_out_deviation

from GetXY_test import x_train, y_train, x_val, y_val, x_test, y_test, out_x_test, out_y_test, long_x_test, long_y_test, outsideExpr, absSum

2025-11-06 20:03:06.686017: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-11-06 20:03:11.477329: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-11-06 20:03:11.477645: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-11-06 20:03:11.480731: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-11-06 20:03:11.480927: I external/local_xla/xla/stream_executor

Epoch 1/200


I0000 00:00:1762455796.346152   25479 service.cc:145] XLA service 0xfffe58004bb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1762455796.346254   25479 service.cc:153]   StreamExecutor device (0): Orin, Compute Capability 8.7
2025-11-06 20:03:16.439309: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-06 20:03:16.723303: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 90300


21/60 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 23.0429

I0000 00:00:1762455797.800917   25479 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - loss: 21.7130 - val_loss: 17.3076
Epoch 2/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 16.3104 - val_loss: 16.1991
Epoch 3/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 15.2697 - val_loss: 15.0167
Epoch 4/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 13.7614 - val_loss: 13.2821
Epoch 5/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 11.5865 - val_loss: 10.6907
Epoch 6/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 8.5600 - val_loss: 7.2572
Epoch 7/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.0771 - val_loss: 3.9051
Epoch 8/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.5072 - val_loss: 2.0082
Epoch 9/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.4236 - val_loss: 1.3743
Epoch 10/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.0784 - val_loss: 1.1357
Epoch 11/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9045 - val_loss: 0.9815
Epoch 12/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.77

/home/capybara/Desktop/matura_project_python/github/matura/FNN1_1.py:246: RuntimeWarning: divide by zero encountered in divide
  relativeError = np.where(np.array(y_test) != 0, deviation.flatten() / np.abs(np.array(y_test)), deviation.flatten())


-5 - -2 - -2
2543
-1.0

Expressions not in x:
2 - -1 + 2
True
1457
5.0
15
-4.0
[-5.    0.75  1.    0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5
  0.5   0.5   0.5 ]


In [2]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
    monitor='val_loss',
    mode = "min"
)

In [3]:
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(x_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)

In [4]:
input_shape = x_train[0].shape
from tensorflow.keras import backend as K

def build_model(hp, input_shape):
    K.clear_session()
    num_neurons = hp.Int("num_neurons", 1, 512)
    num_layers = hp.Int("num_layers", 1, 16)
    dropoutTF = hp.Boolean("dropoutT/F")
    lil_model = keras.Sequential()
    lil_model.add(keras.Input(shape=input_shape))
    lil_model.add(layers.Flatten())
    for i in range(num_layers):
        lil_model.add(layers.Dense(num_neurons)),
        lil_model.add(PReLU())
        if dropoutTF == True:
            lil_model.add(layers.Dropout(0.1))
    lil_model.add(layers.Dense(1, activation='linear'))
    lil_model.compile(optimizer="adam", loss="mse")
    return lil_model
build_model(keras_tuner.HyperParameters(), input_shape)

<Sequential name=sequential, built=True>

In [5]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=lambda hp: build_model(hp, input_shape),    objective="val_loss",
    max_trials=100,
    executions_per_trial=1,
    overwrite=False,
    directory="FNNTuner",
    project_name="tuner",
)

Reloading Tuner from FNNTuner/tuner/tuner0.json


In [6]:
num_epochs = 100
tuner.search(train_dataset, epochs = num_epochs, validation_data = (val_dataset), verbose = 1, callbacks = [])

In [7]:
best_hps = tuner.get_best_hyperparameters()[0]
best_model = build_model(best_hps, input_shape)
best_model.summary()
tuner.results_summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 15)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 345)            │         5,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu (PReLU)                 │ (None, 345)            │           345 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           346 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,211 (24.26 KB)

 Trainable params: 6,211 (24.26 KB)

 Non-trainable params: 0 (0.00 B)

Results summary
Results in FNNTuner/tuner
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 025 summary
Hyperparameters:
num_neurons: 345
num_layers: 1
dropoutT/F: False
Score: 0.0012616724707186222

Trial 000 summary
Hyperparameters:
num_neurons: 348
num_layers: 1
dropoutT/F: False
Score: 0.001294966321438551

Trial 056 summary
Hyperparameters:
num_neurons: 439
num_layers: 4
dropoutT/F: False
Score: 0.0025118887424468994

Trial 020 summary
Hyperparameters:
num_neurons: 16
num_layers: 1
dropoutT/F: False
Score: 0.002628110349178314

Trial 026 summary
Hyperparameters:
num_neurons: 289
num_layers: 2
dropoutT/F: False
Score: 0.002691656118258834

Trial 042 summary
Hyperparameters:
num_neurons: 435
num_layers: 4
dropoutT/F: False
Score: 0.0027040746062994003

Trial 094 summary
Hyperparameters:
num_neurons: 440
num_layers: 4
dropoutT/F: False
Score: 0.002829659730195999

Trial 064 summary
Hyperparameters:
num_neurons: 440
num_layers: 4
dropoutT/F: False
Score: 0.0029

In [8]:
x_test_dataset = tf.data.Dataset.from_tensor_slices(x_test).batch(batch_size)
out_x_test_dataset = tf.data.Dataset.from_tensor_slices(out_x_test).batch(batch_size)
long_x_test_dataset = tf.data.Dataset.from_tensor_slices(long_x_test).batch(batch_size)

In [9]:
best_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=200,
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 19.6619 - val_loss: 16.6378
Epoch 2/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 16.4326 - val_loss: 16.1544
Epoch 3/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 15.7738 - val_loss: 15.6058
Epoch 4/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 14.9615 - val_loss: 14.4177
Epoch 5/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 13.7051 - val_loss: 13.0654
Epoch 6/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 12.1544 - val_loss: 11.2014
Epoch 7/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 10.0626 - val_loss: 9.0231
Epoch 8/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7.7575 - val_loss: 6.5767
Epoch 9/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.4473 - val_loss: 4.4496
Epoch 10/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.4944 - val_loss: 2.8199
Epoch 11/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.2689 - val_loss: 1.8386
Epoch 12/200
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/st

In [10]:
predsInRange = best_model.predict(x_test_dataset)
predsOutRange = best_model.predict(out_x_test_dataset)
predsLongRange = best_model.predict(long_x_test_dataset)

46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [11]:
import numpy as np

reldiffInRange = []
diffInRange = []
safe_y_test = np.where(np.isclose(y_test,0.0), 1.0, y_test)

for i in range(len(y_test)):
    diffInRange.append(abs(y_test[i] - predsInRange[i]))
    reldiffInRange.append(abs(y_test[i] - predsInRange[i])/abs(safe_y_test[i]))
print(len(diffInRange))
print("MAE in Range: ", np.mean(diffInRange))
print("MRE in Range: ", np.mean(reldiffInRange))

diffLongRange = []
for i in range(200, 300):
    diffLongRange.append(np.array(np.abs(long_y_test[i]) - np.array(predsInRange[i])))
    
NEEDdiffLongRange = []
for i in range(len(long_y_test)):
    NEEDdiffLongRange.append(np.array(np.abs(long_y_test[i]) - np.array(predsInRange[i])))
print("MAE longer Expressions: ", np.mean(NEEDdiffLongRange))

diffOutRange = []
for i in range(len(out_y_test)):
    diffOutRange.append(abs(out_y_test[i] - predsOutRange[i]))
safe_out_y_test = np.where(out_y_test == 0, 1, out_y_test)
diff_out_relError = []
for i in range(len(out_y_test)):
    diff_out_relError.append(abs(diffOutRange[i] / safe_out_y_test[i]))
print("MAE out Range: ", np.mean(diffOutRange))
print("MRE out Range: ", np.mean(diff_out_relError))

1457
MAE in Range:  0.040907536
MRE in Range:  0.016460333
MAE longer Expressions:  5.996697
MAE out Range:  2.4127553
MRE out Range:  0.28319752


In [12]:
placeholder = absSum(outsideExpr)
diffOutRange = []
indices_with_placeholder_22 = [i for i, val in enumerate(placeholder) if val == 22] 

for i in indices_with_placeholder_22:
    diffOutRange.append(np.abs(out_y_test[i]-predsOutRange[i]))


In [13]:
meanDiff_InRange = np.mean(diffInRange)
meanDiff_OutRange = np.mean(diffOutRange)
meanDiff_LongRange = np.mean(diffLongRange)
meanDiff_OutRelRange = np.mean(diff_out_relError)



In [14]:
benchmark = 0
benchmark += baseline_deviation / (meanDiff_InRange**2) / 4
print(baseline_deviation / (meanDiff_InRange**2) / 4)

benchmark += baseline_out_deviation / (meanDiff_OutRange**2) / 4
print(baseline_out_deviation / (meanDiff_OutRange**2) / 4)

benchmark += baseline_long_deviation / (meanDiff_LongRange**2) / 4
print(baseline_long_deviation / (meanDiff_LongRange**2) / 4)

benchmark += baseline_relError / (meanDiff_OutRelRange**2) / 4
print(baseline_relError / (meanDiff_OutRelRange**2) / 4)

print(f"Benchmark: {benchmark}")

5.284282296629799
0.28869768329210155
0.10870410318949385
0.15705433625305923
Benchmark: 5.838738419364454
